<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Downloading and Installing pyLiBELa {display-mode: "form"}
%%capture
!pip3 install condacolab
import condacolab
condacolab.install()
! apt-get install openbabel libopenbabel-dev python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
! mamba install -c anaconda py-boost
! git clone https://github.com/alessandronascimento/pyLiBELa.git
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
! make -j4

In [1]:
#@title Importing libraries {display-mode: "form"}
try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!


In [2]:
#@title Getting FDA database {display-mode: "form"}

%%capture
#Ligand list
!rm -rf dbfda.src.txt
source_ligands_file = "http://files.docking.org/catalogs/source/dbfda.src.txt" #@param {type:"string"}
!wget $source_ligands_file


source_ligands = open(r'dbfda.src.txt','r')

ligand_smiles_list = [line.split('\t')[0] for line in source_ligands.readlines()]

source_ligands.close()

!mkdir output

In [36]:
#@title Docking parameters {display-mode: "form"}

import os
import timeit
import numpy as np

Input = PARSER()

# Some default options:
# 1. We are using two processors for Grid calculations;
#

Input.generate_conformers = True;
Input.dock_parallel = True;
Input.parallel_jobs = 2;
Input.write_grids = True
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 60 #@param {type:"number"}
    #atom limit not counting H

# Energy Calculation Parameters:
scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.5 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-10;
Input.min_delta = 1E-5;
Input.dock_min_tol = 1E-10;
search_box = 20.0 #@param {type:"number"}
Input.timeout = 30 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "ln_auglag" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "direct" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
  delta = 2.5 #@param {type:"number"}
  Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
  delta_es = 2.5 #@param {type:"number"}
  Input.deltaij_es6 = pow(delta_es, 6);
  Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;

In [ ]:
#@title Reading molecular files {display-mode: "form"}

#Reading Ligand SMILES


ligand_smiles = "o1ccc2ccccc12" #@param {type:"string"}
#print(ligand_smiles)


Lig2 = Mol2()
if Mol2.parse_smiles(Lig2, Input, ligand_smiles, 'Lig2'):
   print('Ligand SMILES parsed successfully!')

else:
  print('Oops, something wrong with you ligand... \nIt may be way too big or have an unknown atom.')

#Reading reference ligand and receptor from source

lig_src =  'test/lig.mol2.gz' #@param {type:"string"}
rec_src = 'test/rec.mol2.gz' #@param {type:"string"}

REC = Mol2(Input, rec_src)
RefLig = Mol2(Input, lig_src)

print('Receptor and reference ligand parsed successfully!')
print("Receptor has %4d atoms." % REC.N)
print('Reference Ligand has %4d atoms' % RefLig.N)
print('Search ligand has %4d atoms' % Lig2.N)


In [ ]:
#@title Grid Generation and Docking calculation
Writer = WRITER(Input)
Coord = COORD_MC()
HB = FindHB()
Ene  = Energy2(Input)

for i in range(len(REC.residue_pointer)-1):
  HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
HB.find_ligandHB(lig_src, RefLig);
print('The receptor has %5d / %5d HB donors/acceptors around the active site.' % (len(REC.HBdonors), len(REC.HBacceptors)));

Dock = Docker(Writer)
center = Coord.compute_com(RefLig)

print()
start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);
print('Starting energy: %7.3f kcal/mol' % start_energy);
print()
print('Generating grids. This may take a while..')
Grids = Grid(Input, Writer, REC, center)
print('Grids computed!')
grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);
print('Grid original energy: %7.3f kcal/mol' % grid_energy);
print('Grid error: %7.3f %%', abs(100.*(start_energy-grid_energy)/start_energy));
print()
print()
print('Starting docking calculation...')
Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
print('Docking calculation finished!')

Writer.write_box(center, Grids.xbegin, Grids.ybegin, Grids.zbegin, Grids.xend, Grids.yend, Grids.zend)

#colocar pra aparecer energia
#    ultima linha .log
#colocar timeit

#calcular rmsd em relação ao original para todos do SB 2021, em outro Colab
#   aparecer só uns específicos

In [43]:
#@title Preparing Output Files
!obabel -imol2 McLiBELa_dock.mol2.gz -opdb -O lig2.pdb
!obabel -imol2 test/lig.mol2.gz -opdb -O lig_ref.pdb
!obabel -imol2 test/rec.mol2.gz -opdb -O rec.pdb

lig2_pdb = open('lig2.pdb', 'r').read()
lig_ref_pdb = open('lig_ref.pdb', 'r').read()
rec_pdb = open('rec.pdb','r').read()

box_pdb = open('box.pdb','r').read()

def join_pdb(name_file_rec,name_file_lig,name_file_merge):
  !cp $name_file_rec temp_rec.pdb
  !cp $name_file_lig temp_lig.pdb

  !sed -i '/END\|CONECT/d' temp_rec.pdb #tira o end e o conect do arquivo do receptor

  !sed -i 's/A  /B  /g' temp_lig.pdb #troca a cadeia A por B no arquivo do ligante

  !sed -i '/CONECT/d' temp_lig.pdb #tira o conect do arquivo do ligante

  !cat temp_rec.pdb temp_lig.pdb > $name_file_merge #concatena os arquivos

  !rm -rf temp_rec.pdb temp_lig.pdb


join_pdb('rec.pdb','lig_ref.pdb','complex1.pdb')
complex1_pdb = open('complex1.pdb','r').read()

join_pdb('rec.pdb','lig2.pdb','complex2.pdb')
complex2_pdb = open('complex2.pdb','r').read()


In [45]:
#@title Setting view
%%capture
try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol
aminoacids = ['MET', 'THR', 'ASN', 'LYS', 'SER', 'ARG', 'VAL', 'ALA', 'ASP', 'GLU', 'GLY', 'PHE', 'LEU', 'TYR', 'CYS', 'TRP', 'PRO', 'HIS', 'GLN', 'ILE', 'ASH', 'CYX', 'GLH', 'HIE', 'HID', 'HIP','HOH']


In [ ]:
#@title Ligands
view = py3Dmol.view()

#Adding models
view.addModel(lig2_pdb,'pdb') #model 0
view.addModel(lig_ref_pdb,'pdb') #model 1
#view.addModel(box_pdb,'pdb') #something wrong with it

view.setStyle({'model':0},{'stick': {'colorscheme':'yellowCarbon'}})
view.setStyle({'model':1},{'stick': {'colorscheme':'greenCarbon'}})


view.setBackgroundColor('white')
view.zoomTo()
view.show()

In [ ]:
#@title Original complex
view = py3Dmol.view(width=1200,height=800)

#Adding models
view.addModel(complex1_pdb,'pdb') #model 0
view.addModel(rec_pdb,'pdb') #model 1


view.setStyle({'chain':'B'},{'stick': {'colorscheme':'greenCarbon'}})

view.setStyle({'chain':'A'},  {"cartoon":{'arrows':True,'colorscheme':'rasmol','opacity':0.7}})
view.setStyle({ 'chain':'A', 'not':{'resn':aminoacids}}, {'sphere':{}})
view.addStyle({'chain':'A','within':{'distance':'5', 'sel':{'chain':'B'}},'byres':True}, {'stick': {'colorscheme':'skyblueCarbon'}})


view.setBackgroundColor('white')
view.zoomTo({'chain':'B'})
view.show()

In [ ]:
#@title Docking complex
view = py3Dmol.view(width=1200,height=800)

#Adding models
view.addModel(complex2_pdb,'pdb')

view.setStyle({'chain':'B'},{'stick': {'colorscheme':'yellowCarbon'}})

view.setStyle({'chain':'A'},  {"cartoon":{'arrows':True,'colorscheme':'ssPymol','opacity':0.7}})
view.setStyle({ 'chain':'A', 'not':{'resn':aminoacids}}, {'sphere':{}})
view.addStyle({'chain':'A','within':{'distance':'5', 'sel':{'chain':'B'}},'byres':True}, {'stick': {'colorscheme':'skyblueCarbon'}})


view.setBackgroundColor('white')
view.zoomTo({'chain':'B'})
view.show()

In [49]:
#talvez seja bom colocar algo assim no futuro
view = py3Dmol.view(query='pdb:1dc9',linked=False,viewergrid=(2,2))

view.removeAllModels(viewer=(0,0))
view.addModel(lig2_pdb,viewer=(0,0))
view.setStyle({'model':0},{'stick':{'colorscheme':'greenCarbon'}},viewer=(0,0))


view.zoomto()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol